In [4]:
from qiskit import IBMQ
IBMQ.save_account('a1731428f0ac7bf8ff8eb1e356e3dc891ba405dc0f23cad064b518ef3d539eb1d5b5539eeaa5901872fb015414be1b346b435fce4cef3a1a42eb67d4ee6294d0')
import numpy as np
from numpy import linalg
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer
from qiskit import execute, IBMQ, BasicAer
from qiskit.providers.ibmq import least_busy
from tools import set_parameters
from launcher import Launcher
import cmath
import math
parameters = {}
parameters['backend_name'] = 'qasm_simulator'
parameters['shots'] = 100
launcher = Launcher(**{item: parameters[item] for item in ['backend_name', 'shots']})
#ROTATION
def ConditionedRotation(angle, controllingQbit, subjectedQbit, circuit, quantReg):
  
    circuit.u3(angle, 0, 0, quantReg[subjectedQbit])
    circuit.z(quantReg[subjectedQbit])
    circuit.cx(quantReg[controllingQbit], quantReg[subjectedQbit])
    circuit.u3( ( - angle ), 0, 0, quantReg[subjectedQbit])
    circuit.z(quantReg[subjectedQbit])
    
    return
#exp(iA)
def summ(x, y):
    return np.array(x)+np.array(y)
def multm(x, y):
    return np.array(x).dot(np.array(y))
def multmn(z, k):
    return k*np.array(z)
def poweia(x):
    j = complex(0, 1)
    c = multmn(x, j)
    u = [[0 for i in range(n - 1)] for j in range(n)]
    for i in range(n):
        u[i].insert(i, 1)
    a = u
    for i in range(1, 100):
        u = multmn(multm(u, c), 1/i)
        a = summ(a, u)
    return a
#A [2],[1],[1],[2]
n = int(input())
x = []
for i in range(n):
    u = list(map(float, input().split()))
    x.append(u)
#U     
y = poweia(x)
for i in y:
    print(*i, end = '\n')
if n == 2: 
    a = 0.5*cmath.phase(y[0][0]*y[1][1]) ##alf
    d = 2*math.atan2(abs(y[1][0]/y[0][0]),1) + 2*np.pi ##gamma
    b = cmath.phase(y[1][0]/y[0][0])  ##betta
    c = cmath.phase(y[1][1]/y[0][0])-cmath.phase(y[1][0]/y[0][0]) ##delta

#U^2
y_2 = np.array(y).dot(np.array(y))
for i in y_2:
    print(*i, end = '\n')    
if n == 2: 
    a1 = 0.5*cmath.phase(y_2[0][0]*y_2[1][1]) ##alf1
    d1 = 2*math.atan2(abs(y_2[1][0]/y_2[0][0]),1) + 2*np.pi ##gamma1
    b1 = cmath.phase(y_2[1][0]/y_2[0][0])  ##betta1
    c1 = cmath.phase(y_2[1][1]/y_2[0][0])-cmath.phase(y_2[1][0]/y_2[0][0]) ##delta1



# Define the Quantum and Classical Registers
qr = QuantumRegister(5)
cr = ClassicalRegister(5)
# Build the circuits
circuit = QuantumCircuit(qr, cr)
#U
circuit.h(qr[3])
circuit.u1((d - b)/2, qr[4]) #C
circuit.cx(qr[3], qr[4]) #CNOT
circuit.u3(-d/2, 0, -(d + b)/2, qr[4]) #B
circuit.cx(qr[3], qr[4]) #CNOT
circuit.u3(d/2,b, 0, qr[4]) #A
circuit.u1(a, qr[3]) #a

circuit.h(qr[2])
circuit.u1((d1 - b1)/2, qr[4]) #C
circuit.cx(qr[2], qr[4]) #CNOT
circuit.u3(-d1/2, 0, -(d1 + b1)/2, qr[4]) #B
circuit.cx(qr[2], qr[4]) #CNOT
circuit.u3(d1/2,b1, 0, qr[4]) #A
circuit.u1(a1, qr[2]) #a
#FT    
circuit.h(qr[3])
circuit.u1(-(np.pi/4), qr[3])
circuit.cx(qr[2], qr[3])
circuit.u1(-(np.pi/4), qr[3])
circuit.cx(qr[2], qr[3])
circuit.u1(np.pi/2, qr[3])
circuit.u1(-(np.pi/4), qr[2])
circuit.h(qr[2])    
#R
ConditionedRotation(np.pi/4, 2, 1, circuit, qr)
ConditionedRotation(np.pi/8, 3, 1, circuit, qr)
#FT^-1
circuit.h(qr[2]) 
circuit.u1((np.pi/4), qr[2])
circuit.u1(-np.pi/2, qr[3])
circuit.cx(qr[2], qr[3])
circuit.u1((np.pi/4), qr[3])
circuit.cx(qr[2], qr[3])
circuit.u1((np.pi/4), qr[3])
circuit.h(qr[3])
#U^-1

circuit.u1(-a1, qr[2]) #a
circuit.u3(-d1/2,-b1, 0, qr[4]) #A
circuit.cx(qr[2], qr[4]) #CNOT
circuit.u3(d1/2, 0, (d1 + b1)/2, qr[4]) #B
circuit.cx(qr[2], qr[4]) #CNOT
circuit.u1(-(d1 - b1)/2, qr[4]) #C
circuit.h(qr[2])

circuit.u1(-a, qr[3]) #a
circuit.u3(-d/2,-b, 0, qr[4]) #A
circuit.cx(qr[3], qr[4]) #CNOT
circuit.u3(d/2, 0, (d + b)/2, qr[4]) #B
circuit.cx(qr[3], qr[4]) #CNOT
circuit.u1(-(d - b)/2, qr[4]) #C
circuit.h(qr[3])
# meas_qubits - кубиты, использующиеся для томографии. Порядок важен (для [0, 1] и [1, 0] ответ будет разным)
print(launcher.run([circuit], meas_qubits=[1,4]))

2
2 1
1 2
(-0.22484509536615305+0.49129549643388193j) (-0.7651474012342927-0.35017548837401447j)
(-0.7651474012342927-0.35017548837401447j) (-0.22484509536615305+0.49129549643388193j)
(0.27201172505161214+0.31494096431337754j) (0.6881585615987544-0.5943564625123039j)
(0.6881585615987544-0.5943564625123039j) (0.27201172505161214+0.31494096431337754j)


HTML(value="<p style='font-size:16px;'>Job Status: job is actively running </p>")

[array([[ 0.47227212+0.j        ,  0.19838571+0.00324303j,
        -0.02205746+0.04072273j, -0.05602565-0.31333682j],
       [ 0.19838571-0.00324303j,  0.17587982+0.j        ,
        -0.00921346+0.06903533j, -0.00694927-0.14139013j],
       [-0.02205746-0.04072273j, -0.00921346-0.06903533j,
         0.13213197+0.j        , -0.02934699+0.01413148j],
       [-0.05602565+0.31333682j, -0.00694927+0.14139013j,
        -0.02934699-0.01413148j,  0.21971609+0.j        ]])]


In [3]:
circuit.draw()